In [2]:
import json, pandas as pd

rows = []
with open("dataset_dual_ap.jsonl", "r") as fh:
    for line in fh:
        d = json.loads(line)
        row = {"timestamp": d.get("timestamp"), "traffic_label": d.get("traffic_label")}

        for side in ("ap1", "ap2"):
            ap = d.get(side, {}) or {}

            # Métriques iPerf principales
            for k in ("proto", "streams", "udp_rate", "tcp_window",
                      "throughput_bps", "loss_pct", "jitter_ms"):
                row[f"{side}_{k}"] = ap.get(k)

            # TX-Δ (bytes et bps) – gère null/absent
            txd = ap.get("tx_delta") or {}
            for k, v in txd.items():
                if k not in ("timestamp", "traffic_label"):
                    row[f"{side}_txd_{k}"] = v

            # Métriques Wi-Fi globales – gère absent
            wifi = ap.get("wifi") or {}
            for k in ("channel", "frequency_mhz", "bandwidth_mhz", "tx_power_dbm",
                      "cwmin", "cwmax", "noise_floor_dbm", "channel_busy_percent",
                      "nss", "rts_cts",
                      "latency_min_ms", "latency_avg_ms", "latency_max_ms"):
                row[f"{side}_wifi_{k}"] = wifi.get(k)

            # Client: prend le premier s’il existe
            clients = wifi.get("clients") or []
            if clients:
                client = clients[0] or {}
                for ck, cv in client.items():
                    row[f"{side}_client_{ck}"] = cv

        rows.append(row)

df = pd.DataFrame(rows)

display(df.head())
out_path = "dual_ap_flatfinal2.csv"
df.to_csv(out_path, index=False, encoding="utf-8")
print(f"  {len(df)} lignes → '{out_path}' généré.")


,timestamp,traffic_label,ap1_proto,ap1_streams,ap1_udp_rate,ap1_tcp_window,ap1_throughput_bps,ap1_loss_pct,ap1_jitter_ms,ap1_txd_client_mac,...,ap2_wifi_latency_max_ms,ap2_client_client_mac,ap2_client_signal_rssi_dbm,ap2_client_tx_retries,ap2_client_tx_failed,ap2_client_tx_bitrate_mbps,ap2_client_rx_bitrate_mbps,ap2_client_tx_bytes,ap2_client_rx_bytes,ap2_client_mcs_index
0,2025-08-14 13:34:35,RUN_20250814_133412_4,TCP,1,,1M,9.894004e+07,0.000000,NaN,68:54:5a:e2:f2:c0,...,2.704,ac:ed:5c:47:06:cb,-45.0,3.0,3.0,6.0,6.0,111582746.0,383676.0,NaN
1,2025-08-14 13:35:09,RUN_20250814_133442_5,UDP,1,50M,,4.999615e+07,0.000000,0.054515,68:54:5a:e2:f2:c0,...,2.603,ac:ed:5c:47:06:cb,-43.0,450.0,450.0,6.0,6.0,43244620.0,172729.0,NaN
2,2025-08-14 13:36:49,RUN_20250814_133623_8,UDP,1,50M,,4.999614e+07,2.303058,0.252648,68:54:5a:e2:f2:c0,...,2.080,ac:ed:5c:47:06:cb,-46.0,1.0,1.0,6.0,6.0,6637.0,17986.0,NaN
3,2025-08-14 13:37:22,RUN_20250814_133656_9,TCP,1,,512K,1.047876e+08,0.000000,NaN,68:54:5a:e2:f2:c0,...,5.073,ac:ed:5c:47:06:cb,-46.0,0.0,0.0,135.0,27.0,6235.0,18079.0,7.0
4,2025-08-14 13:37:56,RUN_20250814_133729_10,UDP,1,10M,,9.999230e+06,0.000000,0.726885,68:54:5a:e2:f2:c0,...,3.101,ac:ed:5c:47:06:cb,-47.0,0.0,0.0,135.0,40.5,6514.0,15813.0,7.0


✅  297 lignes → 'dual_ap_flatfinal2.csv' généré.


In [3]:
import pandas as pd

# Charger les trois CSV
df1 = pd.read_csv("bd1.csv")
df2 = pd.read_csv("bd2.csv")
df3 = pd.read_csv("bd3.csv")

# Concaténer (coller les lignes les unes sous les autres)
df = pd.concat([df1, df2, df3], ignore_index=True)

# Sauvegarder la base fusionnée
df.to_csv("dataset_merged.csv", index=False)

# Vérifier
print(df.shape)
df.head()


(1041, 66)


,timestamp,traffic_label,ap1_proto,ap1_streams,ap1_udp_rate,ap1_tcp_window,ap1_throughput_bps,ap1_loss_pct,ap1_jitter_ms,ap1_txd_client_mac,...,ap2_wifi_latency_max_ms,ap2_client_client_mac,ap2_client_signal_rssi_dbm,ap2_client_tx_retries,ap2_client_tx_failed,ap2_client_tx_bitrate_mbps,ap2_client_rx_bitrate_mbps,ap2_client_tx_bytes,ap2_client_rx_bytes,ap2_client_mcs_index
0,2025-08-12 14:01:44,RUN_20250812_140125_1,TCP,1,NaN,4M,1.017108e+08,0.000000,NaN,68:54:5a:e2:f2:c0,...,1.884,ac:ed:5c:47:06:cb,-46.0,2432.0,2432.0,135.0,300.0,79083084.0,259462.0,7.0
1,2025-08-12 14:02:50,RUN_20250812_140225_3,TCP,16,NaN,NaN,2.087640e+07,0.000000,NaN,68:54:5a:e2:f2:c0,...,2.227,ac:ed:5c:47:06:cb,-47.0,4558.0,4558.0,390.0,866.7,223226662.0,444056.0,9.0
2,2025-08-12 14:03:23,RUN_20250812_140258_4,UDP,1,10M,NaN,9.999231e+06,0.046339,1.692141,68:54:5a:e2:f2:c0,...,1.937,ac:ed:5c:47:06:cb,-46.0,4879.0,4879.0,135.0,6.0,121632772.0,2707384.0,7.0
3,2025-08-12 14:03:58,RUN_20250812_140330_5,UDP,1,10M,NaN,9.999230e+06,0.023170,1.783586,68:54:5a:e2:f2:c0,...,2.107,ac:ed:5c:47:06:cb,-43.0,2630.0,2630.0,65.0,144.4,63051418.0,295085.0,7.0
4,2025-08-12 14:05:04,RUN_20250812_140439_7,UDP,1,200M,NaN,8.147310e+07,21.565979,0.018009,68:54:5a:e2:f2:c0,...,2.169,ac:ed:5c:47:06:cb,-46.0,499.0,499.0,292.5,12.0,518898178.0,2281874.0,7.0
